In [1]:
import torch
import h5py
import os
import sys
from tqdm import tqdm
sys.path.append('generative_models/')
import sgm
from generative_models.sgm.modules.encoders.modules import FrozenOpenCLIPImageEmbedder
# from generative_models.sgm.models.diffusion import DiffusionEngine
# from omegaconf import OmegaConf

/admin/home-mihirneal/miniconda3/envs/mindeye/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/admin/home-mihirneal/miniconda3/envs/mindeye/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
d1 = torch.device("cuda:0")
d2 = torch.device("cuda:1")
d3 = torch.device("cuda:2")

In [3]:
data_path = "/fsx/proj-fmri/shared/mindeyev2_dataset"
subj=1
data_type = torch.float16

In [4]:
f = h5py.File(f'{data_path}/betas_all_subj0{subj}_fp32.hdf5', 'r')

voxels = f['betas'][:]
print(f"subj0{subj} betas loaded into memory")
voxels = torch.Tensor(voxels).to("cpu").to(data_type)
print("voxels", voxels.shape)
num_voxels = voxels.shape[-1]

# load orig images
f = h5py.File(f'{data_path}/coco_images_224_float16.hdf5', 'r')
images = f['images'][:]
images = torch.Tensor(images).to("cpu").to(data_type)
print("images", images.shape)

subj01 betas loaded into memory
voxels torch.Size([30000, 15724])
images torch.Size([73000, 3, 224, 224])


In [5]:
clip_img_embedder = FrozenOpenCLIPImageEmbedder(
    arch="ViT-bigG-14",
    version="laion2b_s39b_b160k",
    output_tokens=True,
    only_tokens=True,
)
clip_img_embedder.to(d1)

FrozenOpenCLIPImageEmbedder(
  (model): CLIP(
    (visual): VisionTransformer(
      (patchnorm_pre_ln): Identity()
      (conv1): Conv2d(3, 1664, kernel_size=(14, 14), stride=(14, 14), bias=False)
      (patch_dropout): Identity()
      (ln_pre): LayerNorm((1664,), eps=1e-05, elementwise_affine=True)
      (transformer): Transformer(
        (resblocks): ModuleList(
          (0-47): 48 x ResidualAttentionBlock(
            (ln_1): LayerNorm((1664,), eps=1e-05, elementwise_affine=True)
            (attn): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizableLinear(in_features=1664, out_features=1664, bias=True)
            )
            (ls_1): Identity()
            (ln_2): LayerNorm((1664,), eps=1e-05, elementwise_affine=True)
            (mlp): Sequential(
              (c_fc): Linear(in_features=1664, out_features=8192, bias=True)
              (gelu): GELU(approximate='none')
              (c_proj): Linear(in_features=8192, out_features=1664, bias=True)
        

In [13]:
x = torch.randn([3, 224, 224]).cuda()
x = x.unsqueeze(0)
y = clip_img_embedder(x)
x.shape

torch.Size([1, 3, 224, 224])

In [12]:
for i in tqdm(range(images.shape[0])):
    img = images[i].unsqueeze(0).to(d1)
    emb = clip_img_embedder(img)
    full_path = os.path.join("/fsx/proj-fmri/shared/vitBG_embeds/", f"imgt{i+1}_embed.pt")
    torch.save(emb, full_path)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 73000/73000 [33:54<00:00, 35.87it/s]


In [14]:
from torch.utils.data import Dataset
class imgViTBG(Dataset):
    def __init__(self, directory):
        self.directory = directory
        
    def __len__(self):
        return len(os.listdir(self.directory))
    
    def __getitem__(self, idx):
        """
        Load and return the tensor at the given index.
        Args:
        - idx (int): Index of the tensor to be loaded.
        """
        filename = f"imgt{idx+1}_embed.pt"
        file_path = os.path.join(self.directory, filename)
        tensor = torch.load(file_path)
        return tensor

# Usage
directory = '/fsx/proj-fmri/shared/vitBG_embeds/'  # Replace with your directory path
imgemb_dataset = imgViTBG(directory)


In [17]:
x = imgemb_dataset[776]

In [18]:
x.shape

torch.Size([1, 256, 1664])

In [19]:
img = images[776].unsqueeze(0).to(d1)
y = clip_img_embedder(img)

In [20]:
x == y

tensor([[[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]]], device='cuda:0')